# Ethereum Analysis

In [2]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
from chart_studio.plotly import plot, iplot
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from panel.interact import interact
from panel import widgets
import os
import requests
import pandas as pd
import json
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
from poloniex import Poloniex
import time
from datetime import datetime, timezone
import numpy as np
from pathlib import Path
import seaborn as sns
import warnings 
from sklearn.linear_model import LinearRegression

warnings.filterwarnings("ignore")
%matplotlib inline
pd.options.display.float_format = '{:.2f}'.format
load_dotenv()

True

# Import Data
## Ethereum

In [7]:
eth = pd.read_csv("eth.csv",index_col = "date", infer_datetime_format=True, parse_dates=True)
eth_cleaning = eth[['AdrActCnt','PriceUSD']].dropna().reset_index()
eth_cleaning.head(5)

,date,AdrActCnt,PriceUSD
0,2015-08-08,1208.00,1.20
1,2015-08-09,1113.00,1.20
2,2015-08-10,1430.00,1.20
3,2015-08-11,2697.00,0.99
4,2015-08-12,1219.00,1.29


In [10]:
year = eth_cleaning['date'].dt.year
month = eth_cleaning['date'].dt.month
day = eth_cleaning['date'].dt.day
date = {"year":year,"month":month,"day":day}

date_date = pd.DataFrame(date)
eth_file = pd.concat([date_date,eth_cleaning],axis="columns",join="outer")
eth_new_new = eth_file#the original naming will not be changed
eth_test = eth_new_new.drop(['date'],axis=1)#new name for the other operations
eth_new = eth_new_new.drop(['day','date'],axis=1)#new name for the other operations
eth_new_new.head(5)

,year,month,day,date,AdrActCnt,PriceUSD
0,2015,8,8,2015-08-08,1208.00,1.20
1,2015,8,9,2015-08-09,1113.00,1.20
2,2015,8,10,2015-08-10,1430.00,1.20
3,2015,8,11,2015-08-11,2697.00,0.99
4,2015,8,12,2015-08-12,1219.00,1.29


In [12]:
avg_month = eth_new.loc[eth_new['year'] > 2015,:].groupby(["year","month"]).mean()
avg_month.head(5)

AdrActCnt  PriceUSD
year month                     
2016 1        3255.90      1.50
     2        5695.10      4.50
     3       10175.84     11.10
     4       12798.20      8.96
     5       17880.13     11.34

In [14]:
eth_new_new["yearmonth"] = eth_new_new["date"].dt.strftime("%Y-%m")

before_and_after = (eth_new_new
                    .reset_index()
                    .drop("day",axis=1)
                    .loc[eth_new_new['year'] > 2018,:]
                    .groupby(["year","month"])
                    .agg({"AdrActCnt":"mean",
                         "PriceUSD":"mean",
                         "yearmonth":"first"})
                    .reset_index()
                    .drop(["year","month"],axis=1)
                    .set_index("yearmonth")
                    )


#before_and_after = pd.concat([date_con,before_and_after],join="outer",axis="columns").drop(["year","month"],axis=1).set_index
before_and_after.head()

,AdrActCnt,PriceUSD
yearmonth,,
2019-01,259837.61,126.80
2019-02,215373.04,125.57
2019-03,267465.26,135.52
2019-04,303846.47,164.85
2019-05,323094.58,219.83


In [21]:
before_and_after_new = before_and_after.iloc[7::,:]
before_and_after_new.head()
before_and_after_new.hvplot.bar(y=["AdrActCnt","PriceUSD"],
                                rot=90,
                                color=["blue","red"],
                                height=600,width=1250,
                                title="Bar Diagram of the Ethereum Price and Address Action Count",
                                xlabel="Year/MOnth",ylabel="Price/Address Action Count")

:Bars   [yearmonth,Variable]   (value)

In [22]:
adrcount = before_and_after_new.hvplot.line(x="yearmonth",
                                            y="AdrActCnt",
                                            rot=90,
                                            color=["blue","red"],
                                            title="Ethereum Address Count 2019/08 - 2020/11",
                                            xlabel = "Year/Month",ylabel = "Address Action Count")#,
                                 #height=600,width=1250)
pricecount = before_and_after_new.hvplot.line(x="yearmonth",
                                              y="PriceUSD",
                                              rot=90,
                                              color=["blue","red"],
                                              title="Ethereum Price Change 2019/08 - 2020/11",
                                              xlabel = "Year/Month",ylabel="Price in USD")#,
                                 #height=600,width=1250)
adrcount + pricecount

:Layout
   .Curve.I  :Curve   [yearmonth]   (AdrActCnt)
   .Curve.II :Curve   [yearmonth]   (PriceUSD)